[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# Introducción a *Dataframes*.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Intro a Dataframes").getOrCreate()
ct = spark.sparkContext

Los *dataframes* son un concepto compartido entre plataformas como *R*, *Pandas* y *Scala*. Son estructuras tabulares en las que todos los datos de una columna comparten el mismo tipo de datos. Cada columna tiene un título y cada renglón tiene un índice. A la descripción de los tipos de datos de cada columna de un *dataframe* se le conoce como esquema (*schema*).

*PySPark* tiene la capacidad de poder manejar *dataframes* tanto de *Pandas* como de *Spark* e incluso cuenta con una [*API*](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html) que optimiza la interacción entre ambos tipos de *dataframes*.

## Los *dataframes* de *Spark*.

Un *dataframe* de *Spark* es un objeto instanciado de la clase [```pyspark.sql.DataFrame```](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html).

### Creación de *dataframes*.

La función [```spark.createDataFrame()```](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html) permite crear dataframes a partir de objetos que se ingresan como argumentos.


```pyspark
df = spark.createDataFrame(data=<obj>, <títulos columnas>, schema=<esquema>)
```

Donde:

* ```<obj>``` es un objeto que represente una estructura tabular el cual puede ser:
    * Una colección de *Python*.
    * Un *dataframe* de *Pandas*.
    * Un *RDD* de *Spark*.
    
Cabe hacer notar que los *dataframes* de *Spark* se construyen de forma perezosa, por lo que aún cuando sean definidos, estos no serán creados hasta que sean requeridos.
    
Por convención se utiliza el nombre ```df``` para designar undataframe genérico.

### Los objetos ```Row ``` y ```Column```.

Los *dataframes* de *Spark* están compuestos por 2 tipos de objetos.

* Los objetos de tipo ```pyspark.sql.Row``` que representan a cada renglón del *dataframe*.
* Los objetos de tipo ```pyspark.sql.Column``` que representan a cada columna del *dataframe*.

### El método ```df.show()```.

El método ```df.show()``` muestra los primeros ```n``` números de un *dataframe* de *Spark*.

```
df.show(<n>)
```

Donde:

* ```<n>``` es el número de renglones desplegados. El valor por defecto es ```20```.

**Ejemplos:**

* Las siguientes celdas crearán un *dataframe* de *Spark* a partir de un *dataframe* de *Pandas*.

In [ ]:
import pandas as pd

* Se creará el *dataframe* de *Pandas* llamado ```pandas_df```.

In [ ]:
pandas_df = pd.DataFrame({'Dirección':('Sur', 'Norte', 'Sur', 'Este'),
              'Rumbo':('Este', 'Noroeste', 'Norte', 'Norte'),
             'Pasajeros':(12, 24, 32, 5),
             'Documentado':(True, None, True, False) })

In [ ]:
pandas_df

In [ ]:
pandas_df.dtypes

* Se creará el *dataframe* de *Spark* a partir de ```pandas_df```.

In [ ]:
df = spark.createDataFrame(pandas_df)

In [ ]:
df.show()

In [ ]:
df.show(2)

* Las siguientes celdas creará un *dataframe* de *Spark* a partir dde un *RDD* que contiene una colección de objetos ```Row```.

In [ ]:
from pyspark.sql import Row

In [ ]:
rdd = ct.parallelize((Row('Sur', 'Este', '12', True),
                     Row('Norte', 'Noroeste', '24', None),
                     Row('Sur', 'Norte', '32', True),
                     Row('Este', 'Norte', '5', False)))

* La siguiente celda creará un *dataset* a partir del *RDD* y se infgresará como argumento el nombre de cada columna.

In [ ]:
df = spark.createDataFrame(rdd, 
                ['Dirección', 'Rumbo', 'Pasajeros', 'Documentado'])

In [ ]:
df.show()

### Selección de una columna de un *dataframe*.

Es posible acceder a una columna de un *dataframe* usando su nombre o usando su número de índice consecutivo iniciando desde ```0```. 

```
df.<Nombre Columna>
```

```
df[<n>]
```


In [ ]:
df.Rumbo

In [ ]:
df[1]

### El atributo ```df.schema```.

El atributo ```df.schema``` contiene la estructura del *schema* del *dataframe* como una instancia de la clase ```pyspark.sql.types.StructType``` que contiene una colección de instancias de tipo ```pyspark.sql.types.StructField```.

**Ejemplo:**

In [ ]:
df.schema

### El método ```df.printSchema()```.

El método ```df.printSchema()``` despliega una cadena de caractéres describiendo el *schema* del *dataframe*.




In [ ]:
df.printSchema()

## Tipado de *Spark*.

*Spark* cuenta con distintos tipos de datos que extienden a los tipo nativos de *Scala*, *Python* y *R*. El módulo ```pyspark.sql.types``` contiene a todas las clases correspondientes a dichos tipo.

El siguente enlace apunta a la referncia de tipos de datos de *Spark*.

https://spark.apache.org/docs/latest/sql-ref-datatypes.html

* La siguiente celda importará todos los tipos de datos de *PySpark* al entorno de esta *notebook*.

In [ ]:
from pyspark.sql.types import *

## Definición de *schemas* .

### El tipo ```pyspark.api.types.StructField```.

El tipo ```pyspark.api.types.StructField``` permite definir esquemas tanto para describir las columnas de un *dataframe* como para describir *schemas* complejos como los que se pueden encontrar en *YAML* o *JSON*.    
Las estructuras complejas que pueden crearse en los *schemas* son contenidas dentro de un objeto de tipo ```pyspark.api.types.StructType```.

https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.types.StructField.html

El tipo```StructType```.

https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.types.StructType.html

**Ejemplo:**

* La siguiente celda describe un *schema* para el *dataframe* ```df```en le que la columnas ```Pasajeros``` es un entero que va de ```-128```a ```127```.

In [ ]:
schema = StructType([StructField('Dirección', StringType(), True), 
                     StructField('Rumbo', StringType(), True), 
                     StructField('Pasajeros', ByteType(), True), 
                     StructField('Documentado', BooleanType(), True)])

* La siguiente celda creará una *dataframe* con el *schema* correspondiente a ```schema```.

In [ ]:
df = spark.createDataFrame(data=pandas_df, schema=schema)

In [ ]:
df.show()

## Lectura de archivos para *dataframes*.

En el caso de *PySpark*  es psoible leer y crear dataframes a partir de distintos formatos de archivo que describan estructuras tabulares. El atributo  [```spark.read```](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.SparkSession.read.html) contiene una familia de métodos que pueden importar y convertir en *dataframes* diversos tipos de documento.

```
df = spark.read.<método>(<ruta>)
```

### Fuentes de datos compatibles para los *dataframes*.

https://spark.apache.org/docs/latest/sql-data-sources.html

**Ejemplo:**

* La siguiente celda importará los datos del archivo ```data/data_covid.parquet```.

In [ ]:
df = spark.read.parquet('data/data_covid.parquet')

In [ ]:
df

In [ ]:
df.show()

In [ ]:
df.show(1)

* A continuación se usará el método ```df.toPandas()``` para mostrar el *dataframe* ```df``` de mejor manera.

In [ ]:
df.toPandas()

In [ ]:
schema = StructType([StructField('index', DateType(), True), StructField('AGUASCALIENTES', LongType(), True), StructField('BAJA CALIFORNIA', LongType(), True), StructField('BAJA CALIFORNIA SUR', LongType(), True), StructField('CAMPECHE', LongType(), True), StructField('CHIAPAS', LongType(), True), StructField('CHIHUAHUA', LongType(), True), StructField('DISTRITO FEDERAL', LongType(), True), StructField('COAHUILA', LongType(), True), StructField('COLIMA', LongType(), True), StructField('DURANGO', LongType(), True), StructField('GUANAJUATO', LongType(), True), StructField('GUERRERO', LongType(), True), StructField('HIDALGO', LongType(), True), StructField('JALISCO', LongType(), True), StructField('MEXICO', LongType(), True), StructField('MICHOACAN', LongType(), True), StructField('MORELOS', LongType(), True), StructField('NAYARIT', LongType(), True), StructField('NUEVO LEON', LongType(), True), StructField('OAXACA', LongType(), True), StructField('PUEBLA', LongType(), True), StructField('QUERETARO', LongType(), True), StructField('QUINTANA ROO', LongType(), True), StructField('SAN LUIS POTOSI', LongType(), True), StructField('SINALOA', LongType(), True), StructField('SONORA', LongType(), True), StructField('TABASCO', LongType(), True), StructField('TAMAULIPAS', LongType(), True), StructField('TLAXCALA', LongType(), True), StructField('VERACRUZ', LongType(), True), StructField('YUCATAN', LongType(), True), StructField('ZACATECAS', LongType(), True), StructField('Nacional', LongType(), True)])

In [ ]:
df = spark.read.option("header","true").schema(schema).csv('data/data_covid.csv')
df.toPandas()

## Ejemplo de funciones de SQL.

In [ ]:
df.select('index','AGUASCALIENTES').where(df.AGUASCALIENTES >1000).show()

In [ ]:
from datetime import date

In [ ]:
df.select('index','AGUASCALIENTES').where(df.index == date(2020,12,20)).show()

In [ ]:
spark.stop()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>